In [324]:
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
import datetime as dt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import balanced_accuracy_score

### Read realtor data

In [325]:
realtor_2018_data = pd.read_csv(
    Path("realtor_data/2018_realtor_data.csv"))
realtor_2019_data = pd.read_csv(
    Path("realtor_data/2019_realtor_data.csv"))
display(realtor_2018_data)

,sold_date,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size
0,2018-01-02,for_sale,450000.0,3.0,2.0,0.09,"15 William St, North Arlington, NJ, 07031",15 William St,North Arlington,New Jersey,7031.0,NaN
1,2018-01-02,for_sale,234900.0,3.0,2.0,0.87,"185 Gilsum St, Keene, NH, 03431",185 Gilsum St,Keene,New Hampshire,3431.0,1752.0
2,2018-01-02,for_sale,234900.0,3.0,2.0,0.87,"185 Gilsum St, Keene, NH, 03431",185 Gilsum St,Keene,New Hampshire,3431.0,1752.0
3,2018-01-02,for_sale,1245000.0,1.0,1.0,NaN,"57 Reade St Apt 5D, New York, NY, 10007",57 Reade St Apt 5D,New York,New York,10007.0,785.0
4,2018-01-02,for_sale,234900.0,3.0,2.0,0.87,"185 Gilsum St, Keene, NH, 03431",185 Gilsum St,Keene,New Hampshire,3431.0,1752.0
...,...,...,...,...,...,...,...,...,...,...,...,...
25191,2018-12-31,for_sale,389000.0,1.0,1.0,NaN,"62 Park Ter W Apt A26, New York, NY, 10034",62 Park Ter W Apt A26,New York,New York,10034.0,800.0
25192,2018-12-31,for_sale,3995000.0,2.0,3.0,NaN,"200 E 62nd St Apt 14E, New York, NY, 10065",200 E 62nd St Apt 14E,New York,New York,10065.0,1936.0
25193,2018-12-31,for_sale,399000.0,NaN,NaN,1.15,"1 Mayer Ln, Pomona, NY, 10970",1 Mayer Ln,Pomona,New York,10970.0,NaN
25194,2018-12-31,for_sale,3475000.0,2.0,2.0,NaN,"200 Mercer St Apt 4E, New York City, NY, 10012",200 Mercer St Apt 4E,New York City,New York,10012.0,2500.0


In [326]:
csv_files = glob.glob('realtor_data/*')
csv_files

['realtor_data/2019_realtor_data.csv', 'realtor_data/2018_realtor_data.csv']

### Merge realtor data

In [327]:
realtor_data = pd.DataFrame()

for file in csv_files:
            df_temp = pd.read_csv(file)
            realtor_data = realtor_data.append(df_temp)
            

display(realtor_data)

/var/folders/_p/41m0r9cd3bvb4zcy8kyv_4580000gn/T/ipykernel_11904/2010782023.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  realtor_data = realtor_data.append(df_temp)
/var/folders/_p/41m0r9cd3bvb4zcy8kyv_4580000gn/T/ipykernel_11904/2010782023.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  realtor_data = realtor_data.append(df_temp)


,sold_date,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size
0,2019-01-02,for_sale,3480000.0,6.0,7.0,0.19,"116 Bluff Rd, Fort Lee, NJ, 07024",116 Bluff Rd,Fort Lee,New Jersey,7024.0,NaN
1,2019-01-02,for_sale,575000.0,4.0,3.0,0.13,"29 Village Rd, Clifton, NJ, 07013",29 Village Rd,Clifton,New Jersey,7013.0,NaN
2,2019-01-02,for_sale,3480000.0,6.0,7.0,0.19,"116 Bluff Rd, Fort Lee, NJ, 07024",116 Bluff Rd,Fort Lee,New Jersey,7024.0,NaN
3,2019-01-02,for_sale,435000.0,3.0,2.0,0.10,"192 Linwood Ave, Bogota, NJ, 07603",192 Linwood Ave,Bogota,New Jersey,7603.0,NaN
4,2019-01-02,for_sale,349900.0,4.0,2.0,0.24,"16 Dorwin Dr, West Springfield, MA, 01089",16 Dorwin Dr,West Springfield,Massachusetts,1089.0,2002.0
...,...,...,...,...,...,...,...,...,...,...,...,...
25191,2018-12-31,for_sale,389000.0,1.0,1.0,NaN,"62 Park Ter W Apt A26, New York, NY, 10034",62 Park Ter W Apt A26,New York,New York,10034.0,800.0
25192,2018-12-31,for_sale,3995000.0,2.0,3.0,NaN,"200 E 62nd St Apt 14E, New York, NY, 10065",200 E 62nd St Apt 14E,New York,New York,10065.0,1936.0
25193,2018-12-31,for_sale,399000.0,NaN,NaN,1.15,"1 Mayer Ln, Pomona, NY, 10970",1 Mayer Ln,Pomona,New York,10970.0,NaN
25194,2018-12-31,for_sale,3475000.0,2.0,2.0,NaN,"200 Mercer St Apt 4E, New York City, NY, 10012",200 Mercer St Apt 4E,New York City,New York,10012.0,2500.0


### Merge IRS data

### Merge realtor and IRS data on zip_code (left join)

## The final DataFrame we are going to work with!!!

# Prepare the realtor data

In [328]:
#realtor_data.set_index("sold_date", inplace = True)
realtor_data.sort_values(by=["sold_date"], ascending=True, inplace=True)
display(realtor_data)

,sold_date,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size
105,2018-01-02,for_sale,450000.0,3.0,2.0,0.09,"15 William St, North Arlington, NJ, 07031",15 William St,North Arlington,New Jersey,7031.0,NaN
109,2018-01-02,for_sale,2850000.0,4.0,3.0,NaN,"320 W 115th St Unit 1, New York, NY, 10026",320 W 115th St Unit 1,New York,New York,10026.0,1958.0
110,2018-01-02,for_sale,450000.0,3.0,2.0,0.09,"15 William St, North Arlington, NJ, 07031",15 William St,North Arlington,New Jersey,7031.0,NaN
111,2018-01-02,for_sale,2850000.0,4.0,3.0,NaN,"320 W 115th St Unit 1, New York, NY, 10026",320 W 115th St Unit 1,New York,New York,10026.0,1958.0
112,2018-01-02,for_sale,1950000.0,3.0,2.0,NaN,"314 Eighth Ave Unit 2L, Brooklyn, NY, 11215",314 Eighth Ave Unit 2L,Brooklyn,New York,11215.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
24354,2019-12-31,for_sale,1598000.0,3.0,2.0,NaN,"57 L St Apt 10, Boston, MA, 02127",57 L St Apt 10,Boston,Massachusetts,2127.0,1574.0
24353,2019-12-31,for_sale,1598000.0,3.0,2.0,NaN,"57 L St Apt 10, Boston, MA, 02127",57 L St Apt 10,Boston,Massachusetts,2127.0,1574.0
24352,2019-12-31,for_sale,449900.0,5.0,2.0,0.15,"1893 Park Ave, Bridgeport, CT, 06604",1893 Park Ave,Bridgeport,Connecticut,6604.0,1835.0
24362,2019-12-31,for_sale,1598000.0,3.0,2.0,NaN,"57 L St Apt 10, Boston, MA, 02127",57 L St Apt 10,Boston,Massachusetts,2127.0,1574.0


In [329]:
realtor_data = realtor_data.drop(columns=["full_address", "street", "city", "acre_lot", "status", "sold_date"])
display(realtor_data)

,price,bed,bath,state,zip_code,house_size
105,450000.0,3.0,2.0,New Jersey,7031.0,NaN
109,2850000.0,4.0,3.0,New York,10026.0,1958.0
110,450000.0,3.0,2.0,New Jersey,7031.0,NaN
111,2850000.0,4.0,3.0,New York,10026.0,1958.0
112,1950000.0,3.0,2.0,New York,11215.0,NaN
...,...,...,...,...,...,...
24354,1598000.0,3.0,2.0,Massachusetts,2127.0,1574.0
24353,1598000.0,3.0,2.0,Massachusetts,2127.0,1574.0
24352,449900.0,5.0,2.0,Connecticut,6604.0,1835.0
24362,1598000.0,3.0,2.0,Massachusetts,2127.0,1574.0


In [330]:
mean_size = realtor_data["house_size"].mean()
mean_size

2001.3238624665432

In [331]:
realtor_data["house_size"] = realtor_data["house_size"].fillna(mean_size)
display(realtor_data)

,price,bed,bath,state,zip_code,house_size
105,450000.0,3.0,2.0,New Jersey,7031.0,2001.323862
109,2850000.0,4.0,3.0,New York,10026.0,1958.000000
110,450000.0,3.0,2.0,New Jersey,7031.0,2001.323862
111,2850000.0,4.0,3.0,New York,10026.0,1958.000000
112,1950000.0,3.0,2.0,New York,11215.0,2001.323862
...,...,...,...,...,...,...
24354,1598000.0,3.0,2.0,Massachusetts,2127.0,1574.000000
24353,1598000.0,3.0,2.0,Massachusetts,2127.0,1574.000000
24352,449900.0,5.0,2.0,Connecticut,6604.0,1835.000000
24362,1598000.0,3.0,2.0,Massachusetts,2127.0,1574.000000


# Encode the dataset’s categorical variables using OneHotEncoder

In [332]:
# Create a list of categorical variables 
categorical_variables = list(realtor_data.dtypes[realtor_data.dtypes == "object"].index)
display(categorical_variables)

['state']

In [333]:
enc = OneHotEncoder(sparse=False)

In [334]:
encoded_data = enc.fit_transform(realtor_data[categorical_variables])

In [335]:
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names_out(categorical_variables)
)

In [336]:
display(encoded_df)

,state_Connecticut,state_Delaware,state_Maine,state_Massachusetts,state_New Hampshire,state_New Jersey,state_New York,state_Pennsylvania,state_Puerto Rico,state_Rhode Island,state_Vermont,state_Virgin Islands
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
49618,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49619,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49620,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49621,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [337]:
df = realtor_data[['price', 'bed', 'bath', 'zip_code', 'house_size']].dropna()
display(df)

,price,bed,bath,zip_code,house_size
105,450000.0,3.0,2.0,7031.0,2001.323862
109,2850000.0,4.0,3.0,10026.0,1958.000000
110,450000.0,3.0,2.0,7031.0,2001.323862
111,2850000.0,4.0,3.0,10026.0,1958.000000
112,1950000.0,3.0,2.0,11215.0,2001.323862
...,...,...,...,...,...
24354,1598000.0,3.0,2.0,2127.0,1574.000000
24353,1598000.0,3.0,2.0,2127.0,1574.000000
24352,449900.0,5.0,2.0,6604.0,1835.000000
24362,1598000.0,3.0,2.0,2127.0,1574.000000


In [338]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
df.reset_index(inplace=True, drop=True)
encoded_realtor_df = pd.concat([df, encoded_df], axis=1).dropna()
check_nan = encoded_realtor_df.isnull().values.any()
display(encoded_realtor_df)

,price,bed,bath,zip_code,house_size,state_Connecticut,state_Delaware,state_Maine,state_Massachusetts,state_New Hampshire,state_New Jersey,state_New York,state_Pennsylvania,state_Puerto Rico,state_Rhode Island,state_Vermont,state_Virgin Islands
0,450000.0,3.0,2.0,7031.0,2001.323862,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2850000.0,4.0,3.0,10026.0,1958.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,450000.0,3.0,2.0,7031.0,2001.323862,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2850000.0,4.0,3.0,10026.0,1958.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1950000.0,3.0,2.0,11215.0,2001.323862,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46637,1598000.0,3.0,2.0,2127.0,1574.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
46638,1598000.0,3.0,2.0,2127.0,1574.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46639,449900.0,5.0,2.0,6604.0,1835.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46640,1598000.0,3.0,2.0,2127.0,1574.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Apply Logistic Regression

## Split training into testing sets (Create X, or features DataFrame, and create y, or target DataFrame)

In [339]:
y = encoded_realtor_df['price'].values
#np.reshape(y, (1, -1))
display(y)


array([ 450000., 2850000.,  450000., ...,  449900., 1598000.,  919000.])

In [340]:
X = encoded_realtor_df.drop(columns=['price']).values
#X = X[~np.isnan(X)]
#np.isnan(X.any())
np.any(np.isnan(X))
#display(X)

False

In [341]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Use scikit-learn's StandardScaler to scale the features data.


In [342]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create and use a classifier that can predict whether the house sold price price will be higher or lower

In [343]:
logistic_regression_model = LogisticRegression(random_state=1)

# Fit: Train the Model by supplying it with some training it

In [344]:
logistic_regression_model.fit(X_train_scaled, y_train)

/Users/galiy/opt/anaconda3/envs/dev/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=1)

In [354]:
train_pred = logistic_regression_model.predict(X_train_scaled)


In [355]:
balanced_accuracy_score(y_train, train_pred)

0.03234644346609092

# Generate predictions from the model we just fit

In [356]:
# Make a prediction using the testing data
pred = logistic_regression_model.predict(X_test_scaled)

In [357]:
display(pred)

array([155000., 799000., 550000., ..., 575000., 225000., 699000.])

In [350]:
pred_df = pd.DataFrame(pred)
display(pred_df)

,0
0,155000.0
1,799000.0
2,550000.0
3,4595000.0
4,1420000.0
...,...
11656,575000.0
11657,575000.0
11658,575000.0
11659,225000.0


In [351]:
y = pd.DataFrame(y_test)
display(y)

,0
0,725000.0
1,1700000.0
2,318900.0
3,4595000.0
4,1699000.0
...,...
11656,199900.0
11657,429000.0
11658,1399999.0
11659,150000.0


In [352]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, pred)

/Users/galiy/opt/anaconda3/envs/dev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


0.028344661307398104

# Apply the fitted model to the test dataset

In [353]:
results_df = pd.DataFrame({
    "Testing Data Predictions": pred,
    "Testing Data Actual Targets": y_test})
results_df

,Testing Data Predictions,Testing Data Actual Targets
0,155000.0,725000.0
1,799000.0,1700000.0
2,550000.0,318900.0
3,4595000.0,4595000.0
4,1420000.0,1699000.0
...,...,...
11656,575000.0,199900.0
11657,575000.0,429000.0
11658,575000.0,1399999.0
11659,225000.0,150000.0


# Compare each predicted value to its actual value

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, testing_predictions)

# We evaluated the model predictions. If high accuracy (closer to 1) it may ean that there is overfitting which may mean that the model won't perform well on new data it was not trained on

# We can categorize the predictions on higher house prices or lower house prices according to a confusion matrix.